In [1]:
import numpy as np
import pandas as pd
import pandasql as ps

In [2]:
file_name = "gorilla_test_data.xlsx"
meters = pd.read_excel(file_name, sheet_name=0)
forecasts = pd.read_excel(file_name, sheet_name=1)
rates = pd.read_excel(file_name, sheet_name=2)

In [3]:
enhanced_forecasts = pd.merge(meters, forecasts, on="meter_id")

In [4]:
enhanced_forecasts

,meter_id,aq_kwh,exit_zone,date,kwh
0,14676236,28978,EA1,2020-06-01,22.070768
1,14676236,28978,EA1,2020-06-02,19.170720
2,14676236,28978,EA1,2020-06-03,23.555111
3,14676236,28978,EA1,2020-06-04,18.220712
4,14676236,28978,EA1,2020-06-05,14.196134
...,...,...,...,...,...
3407,88357331,484399,SE2,2022-09-27,441.014725
3408,88357331,484399,SE2,2022-09-28,441.512055
3409,88357331,484399,SE2,2022-09-29,437.240657
3410,88357331,484399,SE2,2022-09-30,517.608354


In [5]:
def rate_table_add_end_date():
    start_dates = list(rates.date.unique())
    end_dates = start_dates[1:] + [None]
    list_start_and_end_dates = zip(start_dates, end_dates)
    rates.insert(loc=1, column="end_date", value=rates.date + pd.tseries.offsets.DateOffset(months=6))
    return rates

In [6]:
rates = rate_table_add_end_date()

In [7]:
rates

,date,end_date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
0,2020-04-01,2020-10-01,EA1,0,73200.0,0.2652
1,2020-04-01,2020-10-01,EA1,73200,732000.0,0.1980
2,2020-04-01,2020-10-01,EA1,732000,NaN,0.2875
3,2020-04-01,2020-10-01,EA2,0,73200.0,0.2970
4,2020-04-01,2020-10-01,EA2,73200,732000.0,0.1524
...,...,...,...,...,...,...
1135,2024-10-01,2025-04-01,WM2,73200,732000.0,0.4537
1136,2024-10-01,2025-04-01,WM2,732000,NaN,0.7534
1137,2024-10-01,2025-04-01,WM3,0,73200.0,0.7263
1138,2024-10-01,2025-04-01,WM3,73200,732000.0,0.6109


In [8]:
sqlcode = """
select enhanced_forecasts.meter_id,enhanced_forecasts.aq_kwh,enhanced_forecasts.exit_zone,
enhanced_forecasts.date, enhanced_forecasts.kwh, rates.rate_p_per_kwh, rates.date
from enhanced_forecasts
inner join rates on enhanced_forecasts.exit_zone=rates.exit_zone
where enhanced_forecasts.date >= rates.date and enhanced_forecasts.date < rates.end_date 
and enhanced_forecasts.aq_kwh >= rates.aq_min_kwh and enhanced_forecasts.aq_kwh <= rates.aq_max_kwh
"""

In [9]:
newdf = ps.sqldf(sqlcode, locals())

In [10]:
newdf

,meter_id,aq_kwh,exit_zone,date,kwh,rate_p_per_kwh,date
0,14676236,28978,EA1,2020-06-01 00:00:00.000000,22.070768,0.2652,2020-04-01 00:00:00.000000
1,14676236,28978,EA1,2020-06-02 00:00:00.000000,19.170720,0.2652,2020-04-01 00:00:00.000000
2,14676236,28978,EA1,2020-06-03 00:00:00.000000,23.555111,0.2652,2020-04-01 00:00:00.000000
3,14676236,28978,EA1,2020-06-04 00:00:00.000000,18.220712,0.2652,2020-04-01 00:00:00.000000
4,14676236,28978,EA1,2020-06-05 00:00:00.000000,14.196134,0.2652,2020-04-01 00:00:00.000000
...,...,...,...,...,...,...,...
3407,88357331,484399,SE2,2022-09-27 00:00:00.000000,441.014725,0.3647,2022-04-01 00:00:00.000000
3408,88357331,484399,SE2,2022-09-28 00:00:00.000000,441.512055,0.3647,2022-04-01 00:00:00.000000
3409,88357331,484399,SE2,2022-09-29 00:00:00.000000,437.240657,0.3647,2022-04-01 00:00:00.000000
3410,88357331,484399,SE2,2022-09-30 00:00:00.000000,517.608354,0.3647,2022-04-01 00:00:00.000000


In [11]:
newdf["costs_p"] = newdf["kwh"] * newdf["rate_p_per_kwh"]

In [12]:
result = newdf[["meter_id","kwh", "costs_p"]].groupby("meter_id").sum()
result["costs_pounds"] = result["costs_p"] / 100
result = result.drop(columns="costs_p").round(2)
result

,kwh,costs_pounds
meter_id,,
14676236,28978.0,100.15
34509937,78324.0,275.49
50264822,265667.0,731.24
88357331,484399.0,1433.16
